In [1]:
import xml.etree.ElementTree as ET
import csv
import pandas as pd
import traceback
import sys
import numpy as np


In [2]:
#common variables
CUTTOFF=100000000
LOGAMOUNT=1000000

OUTPUT1="output/output_1.csv"
OUTPUT2="output/output_2.csv"
OUTPUT3="output/output_3.csv"
OUTPUT4="output/output_4.csv"
PROCESSEDFINAL="output/processed_body.csv"
EXISTING_TAGS="output/existing_tags.csv"
TAGSFILE = 'datadumps/Tags.xml'
POSTSFILE = 'datadumps/Posts.xml'

In [13]:
tags_context = ET.iterparse(TAGSFILE, events=("start", "end"),
                       parser=ET.XMLParser(encoding='utf-8'))

In [14]:
# lets look at the tags
_, root = next(tags_context)
for index, item in enumerate(tags_context):
    event, elem = item
    if event == "end" and elem.tag == "row":
        print(elem.attrib)
        print('\n')
        
        if index >1 and index!= 0:
            break
    
    

{'Id': '1', 'TagName': '.net', 'Count': '310793', 'ExcerptPostId': '3624959', 'WikiPostId': '3607476'}


{'Id': '2', 'TagName': 'html', 'Count': '1088688', 'ExcerptPostId': '3673183', 'WikiPostId': '3673182'}




In [15]:
with open(EXISTING_TAGS, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['TAG'])
    _, root = next(tags_context)
    for index, item in enumerate(tags_context):
        event, elem = item
        if event == "end" and elem.tag == "row":
            tag = elem.attrib.get('TagName', 'None')
            csvwriter.writerow([tag])

In [16]:
search_for_tags_df =pd.read_csv(EXISTING_TAGS)
search_for_tags_df = search_for_tags_df.sort_values('TAG')
search_for_tags_df

,TAG
16566,.a
15157,.app
16357,.aspxauth
11751,.bash-profile
15329,.class-file
...,...
36586,zynq
55385,zynq-ultrascale+
29917,zypper
352,NaN


In [17]:
context = ET.iterparse(POSTSFILE, events=("start", "end"),
                       parser=ET.XMLParser(encoding='utf-8'))

In [18]:
# lets look at the data
_, root = next(context)
for index, item in enumerate(context):
    event, elem = item
    if event == "end" and elem.tag == "row":
        print(elem.attrib)
        print('\n')
        tags = elem.attrib.get('Tags', 'None')
        #remove all '<', '>'
        reworked_tags = tags[1:len(tags)-1].split('><')
        print('tags = ', reworked_tags)
        print('\n')
        
    if index >1 and index!= 0:
        break
    
    


{'Id': '4', 'PostTypeId': '1', 'AcceptedAnswerId': '7', 'CreationDate': '2008-07-31T21:42:52.667', 'Score': '735', 'ViewCount': '59537', 'Body': '<p>I want to use a <code>Track-Bar</code> to change a <code>Form</code>\'s opacity.</p>\n<p>This is my code:</p>\n<pre class="lang-cs prettyprint-override"><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n<p>When I build the application, it gives the following error:</p>\n<blockquote>\n<pre class="lang-none prettyprint-override"><code>Cannot implicitly convert type decimal to double\n</code></pre>\n</blockquote>\n<p>I have tried using <code>trans</code> and <code>double</code>, but then the <code>Control</code> doesn\'t work. This code worked fine in a past VB.NET project.</p>\n', 'OwnerUserId': '8', 'LastEditorUserId': '3072350', 'LastEditorDisplayName': 'Rich B', 'LastEditDate': '2021-02-26T03:31:15.027', 'LastActivityDate': '2021-02-26T03:31:15.027', 'Title': 'How to convert a Decimal to a Double in C#?'

In [19]:
def simpleTagCheck(arr):
    return 'is valid' if len(arr) != len(search_for_tags_df.loc[search_for_tags_df['TAG'].
                                                  isin(arr)]) else 'not valid'




In [10]:
search_for_tags_df.loc[search_for_tags_df['TAG'] == 'low-']

,TAG


In [3]:
#platforms
TAGS = ['google-app-maker', 'appian', 'outsystems', 'zoho', 'mendix', 
        'powerapps', 'quickbase', 'vinyl', 'salesforce-lightning', 'github-copilot','Knack','Airtable']

#machine learning
TAGS += ['machine-learning','classification','nlp','anomaly-detection','machine-translation','']

#low-code
TAGS += ['editor','wysiwyg','automation', 'ui-automation','']




COLS = ["Id", "PostTypeId", "AcceptedAnswerId", "ParentId", "CreationDate",
        "DeletionDate", "Score", "ViewCount", "Body", "LastEditDate", "LastActivityDate",
        "Title", "Tags", "AnswerCount", "CommentCount", "FavoriteCount", "ClosedDate",
        "CommunityOwnedDate", "ContentLicense"]

In [21]:
simpleTagCheck(TAGS)

'is valid'

In [22]:
# Extract data by tags
print("Start Extraction")
postcount =0
with open(OUTPUT1, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(COLS)
    _, root = next(context)
    for index, item in enumerate(context):
        event, elem = item
        # prevent duplicates
        if event == "end" and elem.tag == "row":
            tags = elem.attrib.get('Tags', 'None')
            #remove all '<', '>'
            reworked_tags = tags[1:len(tags)-1].split('><')
            for tag in TAGS:
                if tag in reworked_tags:
                    postcount+=1
                    data = []
                    for col in COLS:
                        data.append(elem.attrib.get(col, ''))
                    csvwriter.writerow(data)
                    continue
            elem.clear()
            root.clear()          
        # log progress every 100000 
        if index % LOGAMOUNT == 0 and index != 0:
            print('prossesed', index, 'total_posts_gahtered', postcount)

        if index > CUTTOFF:
            break

print("Extraction Over")

Start Extraction
prossesed 1000000 total_posts_gahtered 587
prossesed 2000000 total_posts_gahtered 1094
prossesed 3000000 total_posts_gahtered 1569
prossesed 4000000 total_posts_gahtered 2042
prossesed 5000000 total_posts_gahtered 2530
prossesed 6000000 total_posts_gahtered 2949
prossesed 7000000 total_posts_gahtered 3397
prossesed 8000000 total_posts_gahtered 3840
prossesed 9000000 total_posts_gahtered 4311
prossesed 10000000 total_posts_gahtered 4736
prossesed 11000000 total_posts_gahtered 5140
prossesed 12000000 total_posts_gahtered 5522
prossesed 13000000 total_posts_gahtered 5988
prossesed 14000000 total_posts_gahtered 6433
prossesed 15000000 total_posts_gahtered 6895
prossesed 16000000 total_posts_gahtered 7341
prossesed 17000000 total_posts_gahtered 7804
prossesed 18000000 total_posts_gahtered 8309
prossesed 19000000 total_posts_gahtered 8786
prossesed 20000000 total_posts_gahtered 9285
prossesed 21000000 total_posts_gahtered 9764
prossesed 22000000 total_posts_gahtered 10224
pr

In [1]:
# clean data
colsSet = set(TAGS)
df = pd.read_csv(OUTPUT1)
df

NameError: name 'TAGS' is not defined

In [24]:
print('REMOVE QUESTIONS WITH WRONG TAGS')
print('Before', len(df))
indicesToRemove = []
for i in range(len(df)):
    tags = df.iloc[i]['Tags'][1:-1].replace('><', ' ').split()
    f = True
    for tag in tags:
        if tag in colsSet:
            f = False
            break
    if f:
        indicesToRemove.append(i)
df = df.drop(index=indicesToRemove)

print('After', len(df))

REMOVE QUESTIONS WITH WRONG TAGS
Before 90855
After 90855


In [25]:
print('\nREMOVING DUPLICATE SCORES')
print('Before', len(df))
df.drop_duplicates('Id',inplace=True)
print('After', len(df))




REMOVING DUPLICATE SCORES
Before 90855
After 85552


In [26]:
print('\nREMOVING QUESTIONS WITH NEGATIVE SCORES')
print('Before', len(df))
df = df[df['Score'] >= 0]
df.to_csv(OUTPUT2, index=False)
print('After', len(df))


REMOVING QUESTIONS WITH NEGATIVE SCORES
Before 85552
After 79345


In [4]:
df = pd.read_csv(OUTPUT2)
df

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense
0,1496,1,1517.0,NaN,2008-08-04T18:51:38.543,NaN,34,9592,<p>For those of us that like to use the graphi...,2019-07-24T14:44:27.110,2019-07-24T14:44:27.110,What are the preferred versions of Vim and Ema...,<macos><vim><emacs><editor>,13,0,6.0,2012-08-03T16:38:59.173,NaN,CC BY-SA 4.0
1,2898,1,2905.0,NaN,2008-08-05T21:42:37.763,NaN,51,129039,<p>Let me preface this question by saying I us...,2012-07-28T00:24:43.417,2021-01-13T14:01:19.757,Text Editor For Linux (Besides Vi)?,<linux><editor>,34,2,19.0,2014-05-28T07:41:58.517,2009-05-03T11:37:34.303,CC BY-SA 2.5
2,10475,1,10479.0,NaN,2008-08-13T23:29:38.707,NaN,26,7864,<p>Since the keyboard is <strong>the</strong> ...,2008-09-11T02:13:53.970,2012-04-01T10:03:18.760,Touch Typing Software recommendations,<editor><touch-typing>,14,1,25.0,2012-03-01T06:12:03.993,NaN,CC BY-SA 2.5
3,10564,1,773973.0,NaN,2008-08-14T01:43:04.270,NaN,595,353209,<p>I'm trying out <strong>Git on Windows</stro...,2016-12-24T17:26:04.737,2021-07-26T08:56:26.523,How can I set up an editor to work with Git on...,<windows><git><cygwin><editor>,35,7,288.0,NaN,NaN,CC BY-SA 2.5
4,11462,1,4500707.0,NaN,2008-08-14T18:30:01.160,NaN,21,19961,"<p>Ok, I've seen a few posts that <em>mention<...",2014-12-09T14:01:56.880,2014-12-09T14:01:56.880,Sharepoint Wikis,<sharepoint><automation><sharepoint-wiki>,17,0,6.0,NaN,NaN,CC BY-SA 3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79340,64002320,1,NaN,NaN,2020-09-22T02:38:02.267,NaN,0,534,<p>Im working on a component where a table is ...,NaN,2020-09-22T02:38:02.267,How do I add fieldinput values to an object in...,<datatables><attributes><salesforce-lightning>...,0,0,NaN,NaN,NaN,CC BY-SA 4.0
79341,64002582,1,NaN,NaN,2020-09-22T03:14:50.670,NaN,0,79,<p>let us say arr1 has size 10 and arr2 is s...,NaN,2020-09-22T04:01:42.873,Is there a way torch.mode can be applied multi...,<python><numpy><machine-learning><pytorch><knn>,1,3,NaN,NaN,NaN,CC BY-SA 4.0
79342,64002652,1,NaN,NaN,2020-09-22T03:24:00.103,NaN,0,83,<p>I want to display records from fieldset val...,NaN,2020-09-22T03:24:00.103,Display records in lightning:dataTable from fi...,<datatable><salesforce><salesforce-lightning>,0,0,NaN,NaN,NaN,CC BY-SA 4.0
79343,64002720,1,NaN,NaN,2020-09-22T03:32:51.170,NaN,0,51,<p>Pretty straightforward... I'm running a Gri...,2020-09-23T10:19:12.950,2020-09-23T10:19:12.950,random forest with sub_balanced returns nan ev...,<python><scikit-learn><classification><random-...,0,4,NaN,NaN,NaN,CC BY-SA 4.0


In [5]:
answerIdDict = {}
for index, row in df.iterrows():
    # access data using column names
    temp = row['AcceptedAnswerId']
    # collect only not nan vlues with indexes
    if not (temp != temp):
        answerIdDict[int(temp)] = index

print('removed nan, reworked total:', len(answerIdDict))



removed nan, reworked total: 35223


In [16]:
context = ET.iterparse(POSTSFILE, events=("start", "end"),
                       parser=ET.XMLParser(encoding='utf-8'))

In [17]:



reacData=[]
with open(OUTPUT2, newline='', encoding='utf-8') as file:
    readData = [row for row in csv.DictReader(file)]

In [18]:
print("Start Processing")
postcount =0
logcounter=LOGAMOUNT


with open(OUTPUT3, "w", newline="", encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    _, root = next(context)
    writer.writerow(COLS + ['AcceptedAnswer','AcceptedAnswerCreationDate'])
    for i, item in enumerate(context):
        event, elem = item

        if logcounter < i < logcounter + LOGAMOUNT:
            logcounter += LOGAMOUNT
            print('prossesed:', i, 'total_answers_gahtered:', postcount)

        if i > CUTTOFF:
            break

        if event == "end" and elem.tag == "row":
            ansId = int(elem.attrib.get('Id', 'None'))
            if ansId in answerIdDict:
                postcount += 1

                ansindex = answerIdDict[ansId]
                readData[ansindex]['AcceptedAnswer'] = elem.attrib.get("Body", '')
                readData[ansindex]['AcceptedAnswerCreationDate'] = elem.attrib.get('CreationDate', '')
                readHeader = readData[ansindex].keys()
                row =[]

                for _, header in enumerate(readHeader):
                    row.append(readData[ansindex][header])

                # print(row)
                # print(type(readData[ansindex]))

                writer.writerow(row)

            elem.clear()
            root.clear()

            
print("End Processing")

Start Processing
prossesed: 1000001 total_answers_gahtered: 374
prossesed: 2000001 total_answers_gahtered: 697
prossesed: 3000001 total_answers_gahtered: 1008
prossesed: 4000001 total_answers_gahtered: 1300
prossesed: 5000001 total_answers_gahtered: 1590
prossesed: 6000001 total_answers_gahtered: 1841
prossesed: 7000001 total_answers_gahtered: 2121
prossesed: 8000001 total_answers_gahtered: 2392
prossesed: 9000001 total_answers_gahtered: 2661
prossesed: 10000001 total_answers_gahtered: 2896
prossesed: 11000001 total_answers_gahtered: 3132
prossesed: 12000001 total_answers_gahtered: 3341
prossesed: 13000001 total_answers_gahtered: 3620
prossesed: 14000001 total_answers_gahtered: 3848
prossesed: 15000001 total_answers_gahtered: 4115
prossesed: 16000001 total_answers_gahtered: 4347
prossesed: 17000001 total_answers_gahtered: 4611
prossesed: 18000001 total_answers_gahtered: 4858
prossesed: 19000001 total_answers_gahtered: 5097
prossesed: 20000001 total_answers_gahtered: 5370
prossesed: 210

In [19]:
df = pd.read_csv(OUTPUT3)
df

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,LastEditDate,...,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense,AcceptedAnswer,AcceptedAnswerCreationDate
0,1496,1,1517.0,NaN,2008-08-04T18:51:38.543,NaN,34,9592,<p>For those of us that like to use the graphi...,2019-07-24T14:44:27.110,...,What are the preferred versions of Vim and Ema...,<macos><vim><emacs><editor>,13,0,6.0,2012-08-03T16:38:59.173,NaN,CC BY-SA 4.0,<p><strong>MacVim</strong> works well and cert...,2008-08-04T19:11:02.140
1,2898,1,2905.0,NaN,2008-08-05T21:42:37.763,NaN,51,129039,<p>Let me preface this question by saying I us...,2012-07-28T00:24:43.417,...,Text Editor For Linux (Besides Vi)?,<linux><editor>,34,2,19.0,2014-05-28T07:41:58.517,2009-05-03T11:37:34.303,CC BY-SA 2.5,<p>Emacs is a wonderful text editor. It has h...,2008-08-05T21:49:12.180
2,10475,1,10479.0,NaN,2008-08-13T23:29:38.707,NaN,26,7864,<p>Since the keyboard is <strong>the</strong> ...,2008-09-11T02:13:53.970,...,Touch Typing Software recommendations,<editor><touch-typing>,14,1,25.0,2012-03-01T06:12:03.993,NaN,CC BY-SA 2.5,"<p><a href=""http://en.wikipedia.org/wiki/Typin...",2008-08-13T23:32:27.560
3,12009,1,12032.0,NaN,2008-08-15T06:08:55.243,NaN,30,30439,<p>How can I pipe the new password to smbpassw...,2009-07-30T18:42:43.350,...,Piping password to smbpasswd,<automation><installation><debian><samba>,7,0,8.0,NaN,NaN,CC BY-SA 2.5,<p>Thanks to Mark I found the answer:</p>\n\n<...,2008-08-15T06:38:59.287
4,12492,1,12534.0,NaN,2008-08-15T17:17:14.823,NaN,84,41385,<p>I use emacs to edit my xml files (nxml-mode...,2008-09-11T02:13:54.640,...,Pretty printing XML files on Emacs,<xml><emacs><editor>,15,0,32.0,NaN,NaN,CC BY-SA 2.5,"<p>I use <a href=""http://www.thaiopensource.co...",2008-08-15T17:47:29.893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35118,63992902,1,63993870.0,NaN,2020-09-21T13:05:45.223,NaN,0,105,<p>I have a convolutional neural network that ...,2020-09-21T16:44:52.343,...,Fitting array to datagen then passing as param...,<python><python-3.x><tensorflow><machine-learn...,1,0,NaN,NaN,NaN,CC BY-SA 4.0,<p>I think at this point you should just make ...,2020-09-21T14:00:35.850
35119,63994413,1,63996417.0,NaN,2020-09-21T14:32:32.487,NaN,1,727,<p>I have tried to implement K Fold Cross Vali...,2020-09-21T15:24:05.847,...,How to use K-Fold Cross Validation For This CNN?,<python><machine-learning><keras><conv-neural-...,2,0,NaN,NaN,NaN,CC BY-SA 4.0,<p>For more flexibility you can use a simple l...,2020-09-21T16:31:08.067
35120,63996947,1,63997974.0,NaN,2020-09-21T17:06:38.423,NaN,2,52,<p>A have two models with the same architectur...,2020-09-21T19:44:25.187,...,Interpolate weights in TensorFlow,<tensorflow><machine-learning>,1,1,NaN,NaN,NaN,CC BY-SA 4.0,<p>That is because of the type mismatch. Both ...,2020-09-21T18:25:37.760
35121,63996893,1,63999665.0,NaN,2020-09-21T17:02:53.757,NaN,1,54,<p>I have the following list:</p>\n<pre><code>...,2020-09-21T18:10:08.590,...,Find first match backwards in a list of lists,<python><regex><list><nlp>,1,3,NaN,NaN,NaN,CC BY-SA 4.0,<p>This can probably somewhat simplified by us...,2020-09-21T20:38:03.957


In [20]:
questionIdDict = {}
for index, row in df.iterrows():
    temp = row['Id']
    if not (temp != temp):
        questionIdDict[int(temp)] = index
        
print('removed nan, reworked total:',len(questionIdDict))

removed nan, reworked total: 35123


In [21]:
questionIdDict

{1496: 0,
 2898: 1,
 10475: 2,
 12009: 3,
 12492: 4,
 12523: 5,
 14410: 6,
 15815: 7,
 16064: 8,
 16991: 9,
 18696: 10,
 17880: 11,
 20533: 12,
 21274: 13,
 24595: 14,
 24610: 15,
 24678: 16,
 26897: 17,
 22326: 18,
 30160: 19,
 21725: 20,
 36533: 21,
 39037: 22,
 39916: 23,
 40943: 24,
 39996: 25,
 41424: 26,
 47167: 27,
 48006: 28,
 49724: 29,
 46454: 30,
 58757: 31,
 59472: 32,
 48782: 33,
 61655: 34,
 61760: 35,
 62328: 36,
 63378: 37,
 64293: 38,
 65800: 39,
 66776: 40,
 66934: 41,
 67729: 42,
 68335: 43,
 60152: 44,
 71336: 45,
 71985: 46,
 73319: 47,
 74625: 48,
 77128: 49,
 80021: 50,
 82726: 51,
 87299: 52,
 76870: 53,
 88399: 54,
 89275: 55,
 90580: 56,
 91834: 57,
 92006: 58,
 91984: 59,
 96066: 60,
 100592: 61,
 93415: 62,
 103155: 63,
 102384: 64,
 103918: 65,
 104878: 66,
 105721: 67,
 107903: 68,
 111683: 69,
 116658: 70,
 120075: 71,
 120862: 72,
 122402: 73,
 122198: 74,
 73927: 75,
 122595: 76,
 127238: 77,
 128625: 78,
 130193: 79,
 131728: 80,
 133626: 81,
 135734: 

In [22]:
context = ET.iterparse(POSTSFILE, events=("start", "end"),
                       parser=ET.XMLParser(encoding='utf-8'))

In [23]:
print("Start Processing")
postcount =0

with open(OUTPUT4, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(COLS)
    _, root = next(context)
    for index, item in enumerate(context):
        event, elem = item
        if event == "end" and elem.tag == "row":
            parentId = int(elem.attrib.get('ParentId', '0'))
            
            if parentId in questionIdDict:
                postcount += 1
                data = []
                for col in COLS:
                    data.append(elem.attrib.get(col, ''))
                csvwriter.writerow(data)
                continue
            
        
            elem.clear()
            root.clear()
        # progress
        if index % LOGAMOUNT == 0 and index != 0:
            print('prossesed:', index, 'total_questions_gahtered:', postcount)
                
        if index > CUTTOFF:
            break
print("End Processing")
 

Start Processing
prossesed: 1000000 total_questions_gahtered: 1915
prossesed: 2000000 total_questions_gahtered: 3072
prossesed: 3000000 total_questions_gahtered: 4074
prossesed: 4000000 total_questions_gahtered: 4936
prossesed: 5000000 total_questions_gahtered: 5711
prossesed: 6000000 total_questions_gahtered: 6333
prossesed: 7000000 total_questions_gahtered: 6996
prossesed: 8000000 total_questions_gahtered: 7606
prossesed: 9000000 total_questions_gahtered: 8167
prossesed: 10000000 total_questions_gahtered: 8665
prossesed: 11000000 total_questions_gahtered: 9136
prossesed: 12000000 total_questions_gahtered: 9591
prossesed: 13000000 total_questions_gahtered: 10138
prossesed: 14000000 total_questions_gahtered: 10595
prossesed: 15000000 total_questions_gahtered: 11145
prossesed: 16000000 total_questions_gahtered: 11615
prossesed: 17000000 total_questions_gahtered: 12114
prossesed: 18000000 total_questions_gahtered: 12557
prossesed: 19000000 total_questions_gahtered: 13021
prossesed: 20000

In [11]:
df = pd.read_csv(OUTPUT3, low_memory=False)

answers_df = df.copy()
answers_df = answers_df[['Id', 'AcceptedAnswer']]
answers_df = answers_df.dropna()
answers_df.columns = ['id', 'raw']
answers_df.insert(1, 'qa', 'a')
answers_df.id = answers_df.id.apply(np.int64)
print(answers_df.head())
print('Total posts', len(answers_df))

questions_df = df.copy()
questions_df = questions_df[['Id', 'Body', 'Title']]
questions_df['titlePlusQuestion'] = questions_df[['Title', 'Body']].apply(lambda x: ' '.join(x), axis=1)
questions_df = questions_df.drop(['Title', 'Body'], axis=1)
questions_df.columns = ['id', 'raw']
questions_df.insert(1, 'qa', 'q')
print(questions_df.head())
print('Total posts:',len(questions_df))

# df = pd.concat([questions_df,answers_df])
# questions_df.head(5).to_html('output/q.html')
# answers_df.head(5).to_html('output/a.html')
# print(len(df))
# df.to_csv(PROCESSEDFINAL, index=False)

      id qa                                                raw
0   1496  a  <p><strong>MacVim</strong> works well and cert...
1   2898  a  <p>Emacs is a wonderful text editor.  It has h...
2  10475  a  <p><a href="http://en.wikipedia.org/wiki/Typin...
3  12009  a  <p>Thanks to Mark I found the answer:</p>\n\n<...
4  12492  a  <p>I use <a href="http://www.thaiopensource.co...
Total posts 35123
      id qa                                                raw
0   1496  q  What are the preferred versions of Vim and Ema...
1   2898  q  Text Editor For Linux (Besides Vi)? <p>Let me ...
2  10475  q  Touch Typing Software recommendations <p>Since...
3  12009  q  Piping password to smbpasswd <p>How can I pipe...
4  12492  q  Pretty printing XML files on Emacs <p>I use em...
Total posts: 35123
